**1. First, let's create a pricing function and check the std **

In [3]:
import numpy as np

In [4]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [31]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
# Comparing the sample mean with the real mean
(default_time.mean(), 1/def_rate)

(10.099238893752359, 10.0)

In [32]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    P=[]
    for i in default_time:
        if (i<mat):
            p_def=recovery*np.exp(-rf_rate*i)
            P.append(p_def)
        else:
            p_mat=np.exp(-rf_rate*mat)
            P.append(p_mat)
    price=np.mean(P)
    return price

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

0.44184995080538775

In [7]:
# Find the mean and std by calling the function 100 times. 
num_sampling=100
p_stat=[]
for i in range(num_sampling):
    p=corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    p_stat.append(p)
    
print(np.mean(p_stat),np.std(p_stat))

0.4404035076968 0.002385019953854281



**2. Now, let's improve the function by reducing the MC variations.**<p>
    (1)Use antithetic method: If U is uniform random variable, so is 1-U<p>
    (2)Also shift the RNs to match the mean, 1/def_rate



In [33]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))#concatenate U with 1-U to make 10000 samples

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [ ]:
# Now include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    if(antithetic):
        U = np.random.uniform(size= int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
        
    P=[]
    for i in default_time:
        if (i<mat):
            p_def=recovery*np.exp(-rf_rate*i)
            P.append(p_def)
        else:
            p_mat=np.exp(-rf_rate*mat)
            P.append(p_mat)
            
    price=np.mean(P)
    return price

In [34]:
corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample)

0.4421195274179043

In [37]:
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
num_sampling2=100
p1_stat2=[]
p2_stat2=[]
p3_stat2=[]
for i in range(num_sampling):
    p1=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,antithetic=True, mean_match=False)
    p1_stat2.append(p1)
    p2=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,antithetic=False, mean_match=True)
    p2_stat2.append(p2)
    p3=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,antithetic=True, mean_match=True)
    p3_stat2.append(p3)
print('With only antithetic feature,the mean and std are',np.mean(p1_stat2),np.std(p1_stat2))
print('With only mean_match feature,the mean and std are',np.mean(p2_stat2),np.std(p2_stat2))
print('With both features,the mean and std are',np.mean(p3_stat2),np.std(p3_stat2))

With only antithetic feature,the mean and std are 0.4405311816437482 0.0015380846751323653
With only mean_match feature,the mean and std are 0.44015552091929044 0.0015478649698159623
With both features,the mean and std are 0.440394770419067 0.0015123452258713357


**3. Finally, what is the analytic value of the corporate bond?<p>How does it compare to your MC result above?**

In [38]:
r=def_rate+rf_rate
real_price=(1-np.exp(-r*mat))*0.3*def_rate/r+np.exp(-r*mat)
print('The analytic value is',real_price)
print('The spread between analystic value and the (i)is',np.abs(real_price-np.mean(p1_stat2)))
print('The spread between analystic value and the (ii)is',np.abs(real_price-np.mean(p2_stat2)))
print('The spread between analystic value and the (iii)is',np.abs(real_price-np.mean(p3_stat2)))

The analytic value is 0.44040907156462505
The spread between analystic value and the (i)is 0.00012211007912316107
The spread between analystic value and the (ii)is 0.00025355064533461036
The spread between analystic value and the (iii)is 1.4301145558059947e-05
